# Make parameter sweep submission for Relaxed Replication model

Author: Juvid Aryaman

Date: 12/01/18

In [1]:
!pwd

/home/juvid/Dropbox/Work/Mit_and_Metabolism/Networks_mtDNA_dynamics/MitoNetworksGenetics/Alternative_control_laws/Param_sweeps/A_relaxed_rep


In [2]:
import numpy as np
import pandas as pd
import pdb

%reload_ext autoreload
%autoreload 2

## Get steady states from ODE integration

In [3]:
dcols = ['param_index','ws','wf','ms','mf']
ss_vals = pd.DataFrame(columns=dcols)
for i in range(121):
    d = pd.read_csv('./Find_det_SS/Data/ode_sol_{}.csv'.format(i))
    ss_vals = ss_vals.append(d.iloc[-1,:][dcols[1:]].set_value('param_index',i))
    

/home/juvid/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


In [4]:
ss_vals['param_index'] = ss_vals['param_index'].astype(int)
ss_vals.reset_index(inplace=True, drop=True)
ss_vals.head()

,param_index,ws,wf,ms,mf
0,0,328.315964,44.684473,140.706842,19.150488
1,1,327.004892,47.374607,140.144954,20.303403
2,2,323.714168,54.132906,138.734643,23.199817
3,3,315.117802,71.069880,135.050487,30.458520
4,4,294.586773,113.073318,126.251474,48.459994


In [5]:
ss_vals[['ws','wf','ms','mf']].sum(axis=1).min(),ss_vals[['ws','wf','ms','mf']].sum(axis=1).max()

(502.25991832379094, 994.07263828742)

## Make Submission

In [6]:
xi = 0.0
beta = 33.12
gamma = 0.03785142857142858
alpha = 1
w_opt = 1000
mu = 0.023
delta = 1.0

nominal_param = [xi,beta,gamma,alpha,w_opt,mu,delta]

In [7]:
work_dir = '/work/ja1109/networks/sing_birth_fus/rel_rep'

Define magnitudes and ratios for ($\beta, \gamma$). Let $M$ be magnitude and $R$ be ratio, then $\beta'=\beta M$ and $\gamma' = \beta' R_0 R$ where $R_0 = \gamma/\beta$. Then this implies $\gamma'=\gamma M R$.

In [8]:
n_points = 11
lg_ls_low_churn = -2
lg_ls_high_churn = 2

lg_ls_low_rat = -2
lg_ls_high_rat = 2

mags = 10**np.linspace(lg_ls_low_churn,lg_ls_high_churn,n_points) # magnitudes of the fusion/fission rate
rats = 10**np.linspace(lg_ls_low_rat,lg_ls_high_rat,n_points) # ratios of the fusion/fission rate

np.log10(mags)

array([-2. , -1.6, -1.2, -0.8, -0.4,  0. ,  0.4,  0.8,  1.2,  1.6,  2. ])

In [9]:
rat_nominal = gamma/beta

In [10]:
n_iters_per_block = 1000 # multiply by NREPEATS in the .c script for the total number of iterations
#n_iters_per_block = 5

In [11]:
initial_seed = 860914 #np.round(np.random.uniform()*1e6).astype(int)
print(initial_seed)

860914


Create first set of parameter sweeps over ($\beta$,$\gamma$) at constant ($\kappa$,$b$).

In [12]:
h_target = 0.3

In [13]:
time_per_job = '20:00:00' # Based on max churn taking 90s for 40 days. For 500 days, 10 iters and x10 safety

In [14]:
def make_submission_string(initial_seed, param_block, n_iters_per_block, ics, new_param):
    submission_string = './relaxed_replication_network.ce {0} {1} {2} $PBS_ARRAY_INDEX'.format(initial_seed,
                                                                                             param_block,
                                                                                             n_iters_per_block)     
    for ic in ics:
        submission_string+=' '+str(ic)

    for p in new_param:
        submission_string+=' {}'.format(p)
    
    submission_string+='\n'

    return submission_string

In [15]:
def make_submission_script(param_block,time_per_job,n_iters_per_block,initial_seed,ics,new_param):
    f = open('submission_{}.pbs'.format(param_block),'w')
    f.write('#!/bin/sh\n')
    f.write('#PBS -N CLC_SW_{}\n'.format(param_block))
    f.write('#PBS -l walltime={}\n'.format(time_per_job))
    f.write('#PBS -l select=1:ncpus=1:mem=1gb\n')
    f.write('#PBS -J 0-{}\n'.format(n_iters_per_block - 1))
    
    f.write('cp $PBS_O_WORKDIR/relaxed_replication_network.ce $TMPDIR\n')
    
    submission_string = make_submission_string(initial_seed, param_block, n_iters_per_block, ics, new_param)
      
    f.write(submission_string) 
    f.write('cp output_{0}_$PBS_ARRAY_INDEX.txt $PBS_O_WORKDIR/p{0:03d}\n'.format(param_block))
    
    # clean up -- have a file number quota
    f.write('sleep 10\n')
    f.write('rm $PBS_O_WORKDIR/CLC_SW_*\n')
    f.close()

In [16]:
param_cols = ['xi', 'beta', 'gamma', 'alpha', 'w_opt', 'mu', 'delta', 
              'net_mag','net_rat',
              'h_init','n_init','ws_init','ms_init','wf_init','mf_init']

In [17]:
param_df = pd.DataFrame(columns=param_cols)

In [18]:
master_script = open('master.sh','w')
for i, mag in enumerate(mags):
    for j, ratio in enumerate(rats):
        
        param_block = len(rats)*i+j
        
        newbeta = beta * mag # beta'
        newgamma = newbeta * rat_nominal * ratio # gamma'
        
        new_param = [k for k in nominal_param]
        new_param[1] = newbeta
        new_param[2] = newgamma
        
        ics = ss_vals[ss_vals['param_index']==param_block][['ws','ms','wf','mf']].round().astype(int).as_matrix()[0]
        
        if np.any(ics < 0):
            raise Exception('Bad ICs!')
                   
        n = np.sum(ics)
        h = (ics[1]+ics[3])/float(n)
        
        param_df = param_df.append(pd.DataFrame([np.hstack((new_param,(mag,ratio,h,n),ics))],columns=param_cols))
        
        make_submission_script(param_block,time_per_job,n_iters_per_block,initial_seed,ics,new_param)

        master_script.write('echo "{}"\n'.format(param_block))
        master_script.write('mkdir -p {0}/p{1:03d}\n'.format(work_dir,param_block))
        master_script.write("qsub submission_{}.pbs\n".format(param_block))
        master_script.write('sleep 0.5\n')
        
            
master_script.close()
param_df.reset_index(inplace=True, drop=True)        

In [19]:
param_df.to_csv('param_sweep_vals.csv',index=False)

In [20]:
param_df

,xi,beta,gamma,alpha,w_opt,mu,delta,net_mag,net_rat,h_init,n_init,ws_init,ms_init,wf_init,mf_init
0,0.0,0.331200,0.000004,1.0,1000.0,0.023,1.0,0.010000,0.010000,0.300188,533.0,328.0,141.0,45.0,19.0
1,0.0,0.331200,0.000010,1.0,1000.0,0.023,1.0,0.010000,0.025119,0.299625,534.0,327.0,140.0,47.0,20.0
2,0.0,0.331200,0.000024,1.0,1000.0,0.023,1.0,0.010000,0.063096,0.300000,540.0,324.0,139.0,54.0,23.0
3,0.0,0.331200,0.000060,1.0,1000.0,0.023,1.0,0.010000,0.158489,0.299456,551.0,315.0,135.0,71.0,30.0
4,0.0,0.331200,0.000151,1.0,1000.0,0.023,1.0,0.010000,0.398107,0.298969,582.0,295.0,126.0,113.0,48.0
5,0.0,0.331200,0.000379,1.0,1000.0,0.023,1.0,0.010000,1.000000,0.300613,652.0,246.0,106.0,210.0,90.0
6,0.0,0.331200,0.000951,1.0,1000.0,0.023,1.0,0.010000,2.511886,0.300390,769.0,163.0,70.0,375.0,161.0
7,0.0,0.331200,0.002388,1.0,1000.0,0.023,1.0,0.010000,6.309573,0.300565,885.0,83.0,36.0,536.0,230.0
8,0.0,0.331200,0.005999,1.0,1000.0,0.023,1.0,0.010000,15.848932,0.300736,951.0,36.0,16.0,629.0,270.0
9,0.0,0.331200,0.015069,1.0,1000.0,0.023,1.0,0.010000,39.810717,0.299694,981.0,15.0,6.0,672.0,288.0
